In [19]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM

In [20]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data

data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [21]:
def setup_data(data):
    """simplify setting up data correctly"""
    #calculate hausmann insturments
    mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
    plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
    exog = np.array( data[['Network Score','Satisfaction Score', 'PPO']])
    
    #concat hausman instr with exog variables
    #hausman_instr = np.concatenate( (exog, np.array(plan_dum)), axis =1 )
    
    hausman_instr = plan_dum
    
    #set up x and y
    y = data[['Inside Good Share','Outside Good Share']]
    x =  data[['Network Score','Satisfaction Score','PPO','Premium']]
    return x,y,hausman_instr

x,y,z =  setup_data(data)

In [80]:
#set up useful global variables 
NMKTS = data['Market_ID'].nunique()
NPLANS = data['Plan_ID'].nunique()
NOBS = data['Plan_ID'].count()
NSIM = 30

theta2 = np.array([3,2,1]) # initialize theta2 for testing purposes
delta = np.ones(NOBS)*(-2)

#print global variables
print NMKTS,NPLANS,NOBS

600 16 3300


In [81]:
#set up v
v = np.genfromtxt('simulations.csv', delimiter=',').reshape(NSIM,3,3300)

In [82]:
def cal_sim_s(data, v, delta, theta2):
    """calculate market share for each simulated consumer"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO']]).transpose()
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1,3300))
    delta  = np.tile( delta  ,(NSIM,1))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() ) 
    
    #sum up between markets
    sim_exp['mkt_id'] = data['Market_ID']
    sum_exp = sim_exp.groupby('mkt_id').sum() 
    sum_exp = pd.merge(data.copy()[['Market_ID']], sum_exp, 
                       right_index=True, left_on = 'Market_ID')
    
    #format so I can broadcast
    sim_exp = np.array(sim_exp).transpose()[:-1]
    sum_exp = np.array(sum_exp).transpose()[1:] + 1
    
    return sim_exp/sum_exp

def cal_s(data, v, delta, theta2):
    """Calculate market share
    Calculates individual choice probability first, then take sum"""
    
    shares = (1./NSIM)*cal_sim_s(data, v, delta, theta2).sum(axis=0)
    return shares


s = cal_s(data, v, delta, theta2)

In [83]:
def cal_delta(data, v, theta2, error = 1e-3, maxiter = 500):
    """Calculate mean utility via contraction mapping"""

    niter = 0
    
    #initialize loop parameters
    delta = np.zeros(NOBS)
    s = cal_s(data, v, delta, theta2)
    diff = np.log(data['Inside Good Share']) - np.log(s)
    
    
    while ((abs(diff).max() > 1e-6) #this is easier to converge
           and (abs(diff).mean() > error) 
           and niter < maxiter):
        
        s = cal_s(data, v, delta, theta2)
        diff = np.log(data['Inside Good Share']) - np.log(s)

        if np.isnan(diff).sum():
            raise Exception('nan in diffs')
            
        delta += diff
        niter += 1

    return delta


delta = cal_delta(data, v, theta2)
print delta.shape
print delta.mean()

(3300,)
2.1218050845


In [84]:
theta1 = np.array([3.56297634,  2.0886027 ,  0.88453571, -2.09580927])
theta2 = np.array([0.21248441, 0.07349874,  1.16047725])

# Part 2 - Elasticities

In [87]:
delta = cal_delta(data, v, theta2)

def cal_price_deriv(data, v, delta, theta1, theta2):

    sim_shares = cal_sim_s(data, v, delta, theta2)
    
    #print sim_shares[1][2]
    sim_shares1 = sim_shares.reshape((NSIM,NOBS,1))
    sim_shares1 = np.tile(sim_shares1,(1,1,NOBS))
    
    print sim_shares1.shape
    
    sim_shares2 = sim_shares.reshape((NSIM,1,NOBS))
    sim_shares2 = np.tile(sim_shares2,(1,NOBS,1))
    
    sim_derivs = sim_shares1*sim_shares2
    print sim_derivs.shape
    
    #sim_derivs = sim_derivs.reshape((NSIM,))
    #alpha = abs(theta1[-1])
    #constant_deriv = alpha*data['Inside Good Share']/data['Premium']
    #shares = (1./NSIM)*(sim_exp/sum_exp).sum(axis=0)
    
    #compute sj * sk in each simulation
    
    #compute (1-sj)sj in each simulation
    
    return sim_shares.sum(axis=0)


#deriv = cal_price_deriv(data, v, delta, theta1, theta2)
#print deriv.mean(), deriv.min(), deriv.max(), deriv.shape

In [89]:
def cal_price_deriv(data, v, delta, theta1, theta2):
    sim_shares = cal_sim_s(data, v, delta, theta2)
    sim_deriv = np.zeros((NOBS,NOBS))
    for sim_share in sim_shares:
        sim_share = sim_share.reshape((NOBS,1))
        sim_deriv = sim_deriv + sim_share.dot(sim_share.transpose())
    return sim_deriv

deriv = cal_price_deriv(data, v, delta, theta1, theta2)
print deriv.shape
print NSIM

(3300, 3300)
30


In [ ]:
def comp_elasticity(shares, prices, alpha):
    #set up matrix skeleton
    own_price = np.identity(len(shares))
    cross_price = 1 - own_price

    #actually calculate elasticity
    cross_elasticity = shares * alpha * prices
    own_elasticity  = -(1-shares) * alpha * prices

    return cross_price*cross_elasticity + own_price *own_elasticity


#agregate elasticities
elasticity = comp_elasticity(shares, prices, alpha)

#average elasticity
avg_shares =  np.array( comp_shares(observ,prices,alpha,beta,
                                    labels=True).groupby(
    'Plan_ID').mean()['fitted_share']).squeeze()
avg_price = np.array( data[['Plan_ID','Premium']].groupby(
    'Plan_ID').mean() ).squeeze()
avg_elasticity = comp_elasticity(avg_shares, avg_price, alpha)

# Part 3 - Marginal Costs

# Part 4 - Counterfactuals

In [ ]:
#will need to compute the change in delta based on the difference in old and new prices
#multiply by alpha
#i.e. delta_new = delta_old - alpha*(p_new - p_old)
#cal_s(data, v, delta, theta2)